In [1]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

import time
import csv

In [2]:
book_list = pd.read_csv('../data/ya_books')
book_list.drop(columns=['Unnamed: 0'], inplace=True)
book_list.head(12)

,author,slug,title
0,J.K. Rowling,3.Harry_Potter_and_the_Sorcerer_s_Stone,Harry Potter and the Sorcerer's Stone (Harry P...
1,Rick Riordan (Goodreads Author),28187.The_Lightning_Thief,The Lightning Thief (Percy Jackson and the Oly...
2,Stephenie Meyer,41865.Twilight,"Twilight (Twilight, #1)"
3,Suzanne Collins,12091570-the-hunger-games,"The Hunger Games (The Hunger Games, #1)"
4,Lois Lowry (Goodreads Author),3636.The_Giver,The Giver
5,Cassandra Clare (Goodreads Author),256683.City_of_Bones,"City of Bones (The Mortal Instruments, #1)"
6,John Green (Goodreads Author),11870085-the-fault-in-our-stars,The Fault in Our Stars
7,Veronica Roth (Goodreads Author),13335037-divergent,"Divergent (Divergent, #1)"
8,Harper Lee,2657.To_Kill_a_Mockingbird,To Kill a Mockingbird
9,Markus Zusak (Goodreads Author),19063.The_Book_Thief,The Book Thief


In [3]:
USERNAME = 'connietiet22@gmail.com'
PASSWORD = 'dsiproject'

### Sign-In Function

In [9]:
def sign_in(username, password):
    driver = webdriver.Chrome('/Users/connie/Downloads/chromedriver 2')
    driver.get('https://www.goodreads.com/user/sign_in')
    email_field = driver.find_element_by_xpath('//*[@id="user_email"]')
    email_field.send_keys(username)
    password_field = driver.find_element_by_xpath('//*[@id="user_password"]')
    password_field.send_keys(password)
    signin_button = driver.find_element_by_xpath('//*[@id="emailForm"]/form/fieldset/div[5]/input')
    signin_button.click()
    return driver

### Retrieve ISBN

In [61]:
def get_isbn(username, password, df, col):
    driver = sign_in(username, password)
    with open('../data/isbns.csv', 'a+') as f:
        writer = csv.writer(f)
        writer.writerow(['book_slug', 'isbn'])
    for book_slug in df[col]:
        isbn = []
        try:
            driver.get(f'https://www.goodreads.com/book/show/{book_slug}')
            WebDriverWait(driver, 10).until(EC.visibility_of_any_elements_located((By.XPATH, '//*[@id="bookDataBox"]/div[2]/div[2]/span/span')))
            source = driver.page_source
            soup = BeautifulSoup(source, 'lxml')
            isbn.append((book_slug, soup.find('span', {'itemprop': 'isbn'}).text))
            with open('../data/book_data/isbns.csv', 'a+') as f:
                writer = csv.writer(f)
                writer.writerows(isbn)
            with open('../data/logfiles/isbn_success.txt', 'a+') as f:
                print(f'{book_slug}', file=f)
            time.sleep(1) 
        except:
            with open('../data/logfiles/isbn_fail.txt', 'a+') as f:
                print(f'{book_slug}', file=f)
    driver.quit()
    return

In [7]:
sample_books = book_list.head(10)

In [59]:
get_isbn(username = USERNAME,
         password = PASSWORD,
         df = sample_books,
         col = 'slug')

### Iterate book_list['slug']

In [6]:
# def get_book_slug(username, password, df, col)
#     driver = sign_in(username, password)
#     for book_slug in df[col]:
#         get_book_reviewers(book_slug, driver)
#     return